In [ ]:
%config InlineBackend.figure_formats = ['png']
import os
import maup
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from gerrychain import Graph
from pcompress import Replay
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from itertools import permutations

In [ ]:
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)

In [ ]:
plans_path = '../../UT/data/plans'
coi_shapefile_path = '../../UT/data/UT-COI-raw'
block_shapefile_path = '../../UT/data/tl_2020_49_tabblock20'
county_shapefile_path = '../../UT/data/Utah_County_Boundaries'
proj = 'EPSG:32612'

In [ ]:
county_gdf = gpd.read_file(county_shapefile_path).to_crs(proj)

In [ ]:
block_gdf = gpd.read_file(block_shapefile_path).to_crs(proj).set_index('GEOID20')

In [ ]:
coi_gdf = gpd.read_file(coi_shapefile_path).to_crs(proj).set_index('PlanID')
coi_gdf = coi_gdf[coi_gdf['MeetsMinim'] == 'Yes']  # remove low-quality submissions

In [ ]:
with maup.progress():
  coi_blocks = maup.intersections(coi_gdf, block_gdf)

In [ ]:
block_areas = dict(block_gdf.geometry.apply(lambda geom: geom.area))

In [ ]:
block_counts = Counter()
for (_, block), geom in coi_blocks.geometry.items():
  if geom.area > 0.01 * block_areas[block]:
    block_counts[block] += 1

In [ ]:
block_gdf['count'] = [block_counts[b] for b in block_gdf.index]

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plt.axis('off')
county_gdf.plot(ax=ax, color='#fffff5', edgecolor='#333')
block_gdf.plot(ax=ax, column='count', cmap='YlOrRd', alpha=0.7, legend=True)
plt.savefig('ut_coi_heatmap.png', dpi=300)
plt.show()